<a href="https://colab.research.google.com/github/Weikang01/Do_something_with_tensorflow_and_keras/blob/master/Take_A_Look_At_Monte_Carlo_Tree_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

本文是对https://www.youtube.com/watch?v=niIaKaWIRX0 所做的笔记
# **Monte Carlo Tree Search 蒙特卡洛树搜索**<br>
> **Monte Carlo Planning 蒙特卡洛规划**

<br>
* 单一状态蒙特卡洛规划：multi-armed bandits多臂赌博机<br><br>
* 上限置信区间策略 Upper Confidence Bound Strategies(UCB)<br><br>
* 蒙特卡洛树搜索 Monte Carlo Tree Search<br><br>

## **多臂赌博机 Multi-Armed Bandits**
> 假设有k个赌博机，用户可以随意摇动某个赌博机的臂膀，赌博机或者吐出，或者不吐任何的硬币。**问题**：当玩了t次赌博机的时候，请问下一次摇动哪一个赌博机，获利可能性最大？

* 单一状态（顾客自己），k种行动（k个摇臂）
* 多臂赌博机是一种序列决策问题，这种问题需要在利用（exploitation）和探索（exploration）之间保持平衡。
 * 利用（exploitation）：保证在过去的决策中得到最佳回报
 * 探索（exploration）：寄希望在未来能获得更大的回报

利用可以理解为：当上一次在n号赌博机上获得了硬币，根据过去的决策经验，你希望再次摇动第n号赌博机。<br>
探索可以理解为：在已经摇动那些赌博机之后，可能还有一些赌博机的臂膀尚未被摇动，你希望在这些未来的可能中获得更加丰厚的回报。<br><br>

* 如果有k个赌博机，这k个赌博机所产生的操作序列为$X_{i,1},X_{i,2},\ldots(i=1,\ldots K)$。在时刻$t=1,2,\ldots$，选择$I_t$个赌博机后，假设可以得到奖励$X_{I_t,t}$，则在n次操作$I_1\ldots I_n$后，可如下定义悔值函数：$$R_n=\max_{i=1,\ldots ,k}\sum^n_{t=1}X_{i,t}-\sum^n_{t=1}X_{I_{t,t}}$$<br>其中$\max_{i=1,\ldots ,k}\sum^n_{t=1}X_{i,t}$表示在$t$次之中，每次都选择回报最大的第$i$个赌博机所获得的回报总和。$\sum^n_{t=1}X_{I_{t,t}}$表示实际之中，摇动$t$次赌博机所获得的回报总和。<p>这个悔值越大，表示实际所做的决策越差，反之，表明决策越好。</p><p>但是，在实际情况之下，悔值函数是不存在的。因为我们无法知道哪一个赌博机会给出最好的回报。</p>



## **上限置信区间策略 Upper Confidence Bound Strategies(UCB)**
> 在多臂赌博机的研究过程中，上限置信区间是一种较为成功的策略学习方法。

* 在UCB方法中，使用$X_{i,T_i(t-1)}$来记录第$i$个赌博机在过去$t-1$时刻内的平均奖赏。则在第$t$个时刻，我们可以选择具有最大上限置信区间的赌博机作为第$t$次所选择的赌博机的臂膀：$$I_t=\max_{i\in\{1,\ldots,k\}}\{\overline{X_{i,T_i(t-1)}}+c_{t-1,T_i(t-1)}\}$$<br>$I_t$表示$t$次所选的赌博机。$\max_{i\in\{1,\ldots,k\}}$意思是现在有$k$个赌博机，使其满足大括号内两者加和最大作为$t$时刻摇动的臂膀。$\overline{X_{i,T_i(t-1)}}$表示第$i$个赌博机在过去$t-1$时刻内的平均回报。很显然，如果在$t-1$时刻，某个赌博机给顾客带来的回报越大，顾客在$t$时刻越容易选择带来了最大平均回报的赌博机。但是，UCB算法告诉我们，仅仅根据过去经验远远不够，我们还需要探索那些虽然现在没有产生最大回报，但是可能产生最大回报的赌博机，在此处由公式第二项所定义。公式第二项$c_{t-1,T_i(t-1)}$的定义：<br>
 * $ c_{t,s}$取值定义如下：$$c_{t,s}=\sqrt{\frac{2lnt}{s}}$$此处$s$表示在$t-1$时刻该赌博机被摇动的次数，也就是说，在过去$t$时刻，这个赌博机被选择了几次。$t$表示在过去一共进行选择的总次数。<br>$c_{t,s}$取值的意义在于，当某个赌博机在过去被选择的次数越少，$s$值越小，则$c_{t,s}$越大，它就越有可能被选择到。$T_i(t)$为在过去时刻（初始时刻到$t$时刻）选择第$i$个赌博机的次数总和。$T_i(t)=\sum^t_{s=1}\prod(I_s=i)$。

* 也就是说，UCB算法会选择具有如下最大值的第$j$个赌博机：$$UCB=\overline{X_j}+\sqrt{\frac{2lin n}{n_j}}\quad\textrm{or}\quad UCB=\overline{X_j}+C\times\sqrt{\frac{2lin n}{n_j}}$$其中$\overline{X_j}$表示第$j$个赌博机在过去时刻带来的平均回报。$n_j$表示过去时间内拉动第$j$个赌博机的次数。$n$是过去拉动赌博机的总次数。$C$是一个平衡因子，其决定着在选择时更偏重探索还是利用。
* 代码实现时，需要先依次摇动所有赌博机，然后再取最大值进行循环。

# **Monte Carlo Tree Search 蒙特卡洛树搜索**<br>
### **1，选择**
* 从根节点R出发，向下递归选择子节点，直至选择到一个叶子节点L。
* 具体来说，通常采用UCB1（平衡因子$C$取值为1）来选择最具潜力的节点。$$UCB=\overline{X_j}+\sqrt{\frac{2lin n}{n_j}}$$如果放到AlphaGo所解决的围棋问题，可以理解为从根节点这个“棋面”出发，依据UCB算法，向下选择过去赢数最多但未来赢数可能更多的子节点。所以，UCB节点做出的，就是根据某个节点在过去的赢面以及是否在过去很少被玩到，但可能带来更大的胜利所进行平衡。

### **2，扩展**
* 如果已经选择到了本选择树的叶子节点L，但其不是终止节点。则随机在其后创建一个未被访问的节点，选择该节点作为后续子节点C。也就是说，在通过UCB算法选择到那些已经展开过了的中间节点并已经找到游戏树的叶子节点之后，随机产生一个未被访问的节点，这个节点就被成为拓展节点。
* **反向传播**
 * 由于计算机计算效率很高，一旦随机选择了某个未被扩展的节点。从该节点出发，对游戏进行模拟，直到博弈的结束。博弈结束之后，我们可以知道这样的路径最终的结果到底是胜利还是失败。于是我们记录下模拟产生的胜利/失败的结果，然后选择导致这个胜利/失败路径上的所有节点，把失败或胜利的信息向上反向回溯回去，让每个节点记录下来。

## **蒙特卡洛树搜索的两种策略学习机制**<br>
* **搜索树策略：**从根节点出发，在已有已经展开的游戏树上，向下递归找到那些潜在的最有潜力的子节点，直到到达本游戏树的叶子节点，然后随机产生一个未被访问的节点。（即MCTS中选择和扩展两个步骤）。搜索树策略需要在利用和探索之间保持平衡。
* **模拟策略：**从拓展出来的结点出发，进行游戏的仿真，一直到游戏的结束，来判定：依照这样的路径来拓展之后，游戏的策略到底是赢还是输。<br><br>

#### **围棋博弈的例子**
* 用一棵树模型来表示围棋的棋面状况，黑棋先手，假设根节点目前由UCB1算法计算出来的值为A/B，其中A是黑棋胜利的次数，B为该节点的总访问次数。<br>如假设此局根节点的A/B值为12/21。其中12表示黑棋在此状态之中最终胜利的次数（同时也是白棋失败的次数），21为经过此节点的总次数。
* 假设从根节点出发，<b>此时由黑棋行棋</b>。有三个可能的子节点，其值分别为7/10、5/8、0/3。算法先进行选择操作，即使用UCB1算法计算最有潜力的后续节点：
 * 7/10对应的最终奖励值为$\frac{7}{10}+\frac{\sqrt{log(21)}}{10}=1.252$
 * 5/8对应的最终奖励值为$\frac{5}{8}+\frac{\sqrt{log(21)}}{8}=1.243$
 * 0/3对应的最终奖励值为$\frac{0}{3}+\frac{\sqrt{log(21)}}{3}=1.007$

 由此可见，黑棋会选择局面7/10进行游戏。
* 进入节点7/10，<b>此时轮到白棋行棋</b>。假设该节点此时下面有两个节点，分别为2/4和5/6，则继续利用UCB1算法进行计算：
 * 2/4对应的最终奖励值为$1-\frac{2}{4}+\frac{\sqrt{log(21)}}{4}=1.372$
 * 5/6对应的最终奖励值为$1-\frac{5}{6}+\frac{\sqrt{log(21)}}{6}=0.879$

 由此可见，白棋会选择局面2/4进行游戏。

* 进入节点2/4，<b>轮到黑棋行棋</b>。此时该节点下有子节点1/3和1/1，分别利用UCB1算法进行计算：
 * 1/3对应的最终奖励值为$\frac{1}{3}+\frac{\sqrt{log(21)}}{3}=1.341$
 * 1/1对应的最终奖励值为$\frac{1}{1}+\frac{\sqrt{log(21)}}{1}=2.745$

 由此可见，黑棋会选择局面1/1进行游戏。
* 假设游戏树已经到达了叶子节点。此时随机扩展一个新节点。由于该节点未被访问，所以初始化为0/0，接着在该节点之下进行模拟。<br>假设经过一系列仿真行棋之后，最终白棋获胜。根据仿真结果来更新路径上每一个节点的A/B值，该节点的A/B值更新为0/1，并向上回溯到所有父节点。<br><br>

**在有限的时间之内，算法会一直重复此过程，并找到在规定时间之内计算出来的奖励值最优的节点作为真正决策的节点。**

#### **Pseudo-code of UCT & UCTSearch**

> **Terms**
* $S$：状态集States
* $A_{(s)}$：在状态s能够采取的有效行动Action的集合
* $s(v)$：节点v所代表的状态
* $a(v)$：采取的行动导致到达节点v
* $f\quad\quad S\times A \rightarrow S$：状态转移函数
* $N(v)$：节点v被访问的次数
* $Q(V)$：节点v所获得的奖励值
* $\Delta(v,p)$：玩家p选择节点v所得到的奖励值<br>

<pre>
<b>function</b> UCTSearch(s_0):
  create root node v_0 with state s_0
  <b>while</b> within computational budget <b>do</b>:
    v_l <= TreePolicy(v_0)
    delta <= DefaultPolicy(s(v_l))
    Backup(v_l,delta)
  <b>return</b> a(BestChild(v_0,0))
</pre>
<pre>
<b>function</b> TreePolicy(v):
  <b>while</b> v is non-terminal <b>do</b>:
    <b>if</b> v not fully expanded <b>then</b>:
      <b>return</b> Expand(v)
    <b>else</b>:
      v <= BestChild(v,Cp)  # C是UCB的平衡因子
  <b>return</b> v
</pre>
<pre>
<b>function</b> Expand(v):
  choose a belongs to untried actions set A(s(v))
  add a new child v` to v:
    with s(v`) <= f(s(v),a)
    and a(v`) <= a
  <b>return</b> v`
</pre>
<pre>
<b>function</b> DefaultPolicy(s):
  <b>while</b> s is non-terminal <b>do</b>:
    choose a belongs to A(s) uniformly at random
    s <= f(s,a)
  <b>return</b> reward of state s
</pre>
<pre>
<b>function</b> Backup(v, delta):
  <b>while</b> v is not null <b>do</b>:
    N(v) <= N(v) + 1
    Q(v) <= Q(v) + delta(v,p)
    v <= parent of v
</pre>
<pre>
<b>function</b> BestChild(v,c):
  <b>return</b> argmax(v` belongs to children of v)(Q(v`)/N(v`)+c*sqrt(2*ln(N(v)/N(v`)))  # UCB算法
</pre>